In [1]:
import pandas as pd
import numpy as np

In [2]:
from os import listdir
from os.path import isfile, join

path = "../data connectors/google trends/normalized_data_daily/"
files = [f for f in listdir(path) if isfile(join(path, f))]

In [3]:
files

['normalized_bitcoin_2010-01-01-2020-03-08.csv',
 'normalized_ethereum_2016-01-01-2020-03-08.csv',
 'normalized_bitfinex_2016-01-01-2020-03-08.csv',
 'normalized_coinbase_2016-01-01-2020-03-08.csv',
 'normalized_binance_2016-07-01-2020-03-08.csv']

In [4]:
trends_col = 'trend'

feature_names = [trends_col,'sma10','sma21']

for file in files:
    data = pd.DataFrame()
    #read data
    file_path = path+file
    d = pd.read_csv(file_path)
    d['timestamp'] = [pd.Timestamp(t) for t in d['date']]
    d.rename(columns={'normalized_value':'trend'},inplace=True)
    kw = d['keyword'].values[0]
    
    #daily average
    d['day'] = [ts.strftime('%Y-%m-%d') for ts in d['timestamp'].to_list()]
    d = d.groupby(by=['day']).mean()
    d.index = [pd.Timestamp(ts) for ts in d.index.to_list()]
    d['day'] = d.index
    d['keyword'] = kw
    
    x = d[['keyword','day',trends_col]].copy()
    
    #generate sma of trend
    for f in feature_names[1:]:
        x[f] = x.rolling(window=int(f[-2:]))['trend'].mean()
    
    for feature in feature_names:
        x['log_'+feature] = np.log(x[feature])
        x['diff_'+feature] = x[feature].diff()
        x['log_diff_'+feature] = x['log_'+feature].diff()
        x['diff_ratio_'+feature] = (x['diff_'+feature]/x[feature])*100
    
    data = pd.concat([data,x],ignore_index=True)
    
    data.to_csv("generated_trends_data/"+kw+"_daily.csv",index=False)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/algorithms.py:1926: RuntimeWarning: invalid value encountered in subtract
  out_arr[res_indexer] = arr[res_indexer] - arr[lag_indexer]


In [5]:
data

,keyword,day,trend,sma10,sma21,log_trend,diff_trend,log_diff_trend,diff_ratio_trend,log_sma10,diff_sma10,log_diff_sma10,diff_ratio_sma10,log_sma21,diff_sma21,log_diff_sma21,diff_ratio_sma21
0,binance,2016-07-01,0.000000,NaN,NaN,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,binance,2016-07-02,0.000000,NaN,NaN,-inf,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,binance,2016-07-03,0.000000,NaN,NaN,-inf,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,binance,2016-07-04,0.000000,NaN,NaN,-inf,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,binance,2016-07-05,0.000000,NaN,NaN,-inf,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,binance,2021-03-04,13.521364,15.092766,18.654958,2.604271,-0.730885,-0.052644,-5.405405,2.714216,-0.803973,-0.051898,-5.326877,2.926112,-0.417648,-0.022141,-2.238806
1708,binance,2021-03-05,11.328711,14.398426,18.254712,2.427340,-2.192654,-0.176931,-19.354839,2.667119,-0.694340,-0.047097,-4.822335,2.904423,-0.400246,-0.021689,-2.192564
1709,binance,2021-03-06,12.059595,13.777174,17.906672,2.489861,0.730885,0.062520,6.060606,2.623013,-0.621252,-0.044106,-4.509284,2.885173,-0.348040,-0.019250,-1.943635
1710,binance,2021-03-07,12.790480,13.448276,17.645641,2.548701,0.730885,0.058841,5.714286,2.598851,-0.328898,-0.024162,-2.445652,2.870489,-0.261030,-0.014685,-1.479290


In [11]:
for c in data.columns:
    print(c,data[c].shape,data[c].dropna().shape)

keyword (1712,) (1712,)
day (1712,) (1712,)
trend (1712,) (1712,)
sma10 (1712,) (1703,)
sma21 (1712,) (1692,)
log_trend (1712,) (1712,)
diff_trend (1712,) (1711,)
log_diff_trend (1712,) (1186,)
diff_ratio_trend (1712,) (1186,)
log_sma10 (1712,) (1703,)
diff_sma10 (1712,) (1702,)
log_diff_sma10 (1712,) (1304,)
diff_ratio_sma10 (1712,) (1304,)
log_sma21 (1712,) (1692,)
diff_sma21 (1712,) (1691,)
log_diff_sma21 (1712,) (1350,)
diff_ratio_sma21 (1712,) (1350,)
